## Advent of code 2023 day 21-25
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv environment (which is set up with pypy3.10-v7.3.13-win64)
# to activate it from a git bash shell: source .venv/Scripts/activate

import collections
import itertools
import functools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import dataclasses

import sortedcontainers

import zio

In [ ]:
# version check and timestamp
# NB the timestamp supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}')

In [ ]:
# 2023 day 23 part 1
# start_ts=1703324076
# mv ~/Downloads/input* data_src/2023-day-23-input.txt
# big input file looks like: a big map
# idea: part 1 parse as a text map, then BFS for max steps

sample2='''
#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#
'''

def walk_long(lines):
    '''determine longest path and return it'''
    board={} # maps (x,y) to max steps to reach that cell
    sx=lines[0].index('.'); sy=0
    ex=lines[-1].index('.'); ey=len(lines)-1
    intodo=[ (sx,sy,0, {(sx,sy),}), ] # tuples of (x,y,steps-so-far, visited)
    outtodo=[]
    w=len(lines[0]); h=len(lines)

    def add_step(ox, oy, nx, ny, newsteps, oldvisit):
        if nx<0 or ny<0 or nx>=w or ny>=h:
            return
        if lines[ny][nx]=='#':
            return
        if (nx,ny) in oldvisit:
            return
        newvisit=oldvisit.copy()
        newvisit.add( (nx,ny) )
        intodo.append( (nx, ny, newsteps, newvisit) )

    while True:
        if len(intodo)<1:
            break
        outtodo.extend(intodo)
        intodo.clear()
        for todotup in outtodo:
            x,y,oldsteps,oldvisit=todotup
            xy=(x,y)
            if xy not in board or oldsteps>board[xy]:
                board[xy]=oldsteps
                #if x==ex and y==ey:
                #    print(f'score updated to {oldsteps}')
            c=lines[y][x]
            if c=='.' or c=='>':
                add_step(x, y, x+1, y, oldsteps+1, oldvisit)
            if c=='.' or c=='<':
                add_step(x, y, x-1, y, oldsteps+1, oldvisit)
            if c=='.' or c=='v':
                add_step(x, y, x, y+1, oldsteps+1, oldvisit)
            if c=='.' or c=='^':
                add_step(x, y, x, y-1, oldsteps+1, oldvisit)
        outtodo.clear()
    return board, board.get( (ex,ey) )

sample1=open('data_src/2023-day-23-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
board, score=walk_long(lines)
print(f'part 1: {score=}')

# part 1: 2326

#lines=[s.replace('<', '.').replace('>', '.').replace('v', '.').replace('^', '.') for s in lines ]
#board, score=walk_long(lines)
#print(f'part 2: {score=}')

In [ ]:
# 2023 day 23 part 2 a
# idea: start from the middle (every spot that's open on the diagonal), get the shortest path
# to eiter start or end, then occupy it, create longest paths and pick the one to the opposite start/end,
# occupy that, then instead of the initial shortest path recreate the longest path to the initial start/end,
# then of all those options take the longest
# this should work as long as the longest path from the diagonal spot of the real longest path
# doesn't cross the shortest path to the other side
# (and it's faster than the previous method because the longest path search space is smaller than when starting from
# one corner, in particular when occupying part of it with the shortest path to the opposite corner)

def shortest_path(lines, sx, sy, ex, ey):
    '''find and return shortest path from sx,sy to ex,ey
    (the resulting list does not contain sx,sy at the start but does contain ex,ey at the end)'''
    w=len(lines[0]); h=len(lines)
    board={} # maps x,y to shortest distance from sx,sy
    todo={ (sx,sy, 0) } # tuples of x,y,distance-so-far

    def add_step(nx,ny,newdist):
        '''add a BFS todo step if valid'''
        if nx<0 or ny<0 or nx>=w or ny>=h:
            return
        if lines[ny][nx]=='#':
            return
        todo.add( (nx,ny,newdist) )

    def gather_path(x, y, path):
        '''based on BFS board back-gather (one of) the shortest path(s) using DFS'''
        if x==sx and y==sy:
            return path
        path.insert(0, (x,y))
        olddist=board[(x,y)]
        for nx,ny in [(x+1,y), (x-1,y), (x,y+1), (x,y-1)]:
            if board.get( (nx,ny) ) == olddist-1:
                p2=gather_path(nx, ny, path)
                if p2 is not None:
                    return p2
        path.pop(0)
        return None

    while len(todo)>0:
        xyd=todo.pop()
        x,y,olddist=xyd
        xy=(x,y)
        if xy not in board or board[xy] >olddist:
            board[xy]=olddist
            add_step(x+1, y, olddist+1)
            add_step(x-1, y, olddist+1)
            add_step(x, y+1, olddist+1)
            add_step(x, y-1, olddist+1)
    if (ex,ey) not in board:
        return None
    # now gather the path, starting from ex,ey
    return gather_path(ex, ey, [])

In [ ]:
# 2023 day 23 part 2 b

def longest_path(lines, sx, sy, ex, ey, blocked0):
    '''determine longest path and return it, uses DFS, avoids blocked squares,
    (the resulting list does not contain sx,sy at the start but does contain ex,ey at the end)'''
    # implementation assumes set is maintained in insertion order
    w=len(lines[0]); h=len(lines)
    blocked=blocked0.copy() # block off more cells to make lon_path simpler / faster
    for x in range(-1, w+1):
        for y in range(-1, h+1):
            if x<0 or y<0 or x>=w or y>=h or lines[y][x]=='#':
                blocked.add( (x,y) )
    epath=None # longest path to end

    def lon_path(x, y, path):
        nonlocal epath
        xy=(x,y)
        if xy in path or xy in blocked:
            return
        path.add(xy)
        if x==ex and y==ey:
            if epath is None or len(path)>len(epath):
                epath=path.copy()
                path.remove(xy)
                return
        lon_path(x+1, y, path)
        lon_path(x-1, y, path)
        lon_path(x, y+1, path)
        lon_path(x, y-1, path)
        path.remove(xy)

    lon_path(sx, sy, set())
    if epath is not None:
        epath.discard( (sx,sy) )
    return epath

def get_diagonals(lines):
    '''return open cells on the diagonal (if square) or horizontal separator (if non-square)'''
    w=len(lines[0]); h=len(lines)
    if w==h:
        res=[ (x,h-1-x) for x in range(w) ]
    else:
        res=[ (x, h//2) for x in range(w) ]
    res=[ xy for xy in res if lines[xy[1]][xy[0]]!='#' ]
    return res

In [ ]:
# 2023 day 23 part 2 c

sample2='''
#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#
'''

sample1=open('data_src/2023-day-23-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
sx=lines[0].index('.'); sy=0
ex=lines[-1].index('.'); ey=len(lines)-1
diags=get_diagonals(lines)
print(f'{sx=}, {sy=}, {ex=}, {ey=}, {len(diags)=}')

bestlen=0
for dxy in diags:
    dx,dy=dxy
    for it in [1,2]: # 1 is starting with shortest path to sx,sy, 2 same to ex,ey
        sp1=shortest_path(lines, dx, dy, sx, sy) if it==1 else shortest_path(lines, dx, dy, ex, ey)
        if sp1 is not None:
            lp2=longest_path(lines, dx, dy, ex, ey, set(sp1)) if it==1 else longest_path(lines, dx, dy, sx, sy, set(sp1))
            if lp2 is not None:
                lp3=longest_path(lines, dx, dy, sx, sy, set(lp2)) if it==1 else longest_path(lines, dx, dy, ex, ey, set(lp2))
                if lp3 is not None:
                    tplen=len(lp2)+len(lp3)
                    bestlen=max(bestlen, tplen)
                    print(f'{time.ctime()} found {dxy=}, {it=}, {tplen=}, {bestlen=}')
print(f'part 2: score={bestlen}')

# part 2: 6574 (after 219 minutes, although the final answer was already found after ca. 150 minutes)

In [ ]:
# 2023 day 21 part 1
# start_ts=1703154960
# mv ~/Downloads/input* data_src/2023-day-21-input.txt
# big input file looks like: big map
# idea: part 1 parse as text map, then BFS keeping track per cell of the steps you can take to reach it,
#  with cutoff above 64

sample2='''
...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
'''

def find_sx_sy(lines):
    '''find the S'''
    for y,row in enumerate(lines):
        for x,c in enumerate(row):
            if c=='S':
                return x,y
    assert False

def take_steps(lines, maxsteps, sx, sy):
    '''determine which cells can be reached in how many steps, up to maxsteps'''
    board={} # maps (x,y) to set of steps that can be used to reach that cell
    todo={ (sx,sy,0), } # tuples of (x,y,steps-so-far)
    w=len(lines[0]); h=len(lines)

    def add_step(nx, ny, newsteps):
        if newsteps>maxsteps:
            return
        if nx<0 or nx>=w or ny<0 or ny>=h:
            return
        if lines[ny][nx]=='#':
            return
        todo.add( (nx, ny, newsteps) )

    while len(todo)>0:
        todotup=todo.pop()
        x,y,oldsteps=todotup
        cs=board.setdefault( (x,y), set())
        if oldsteps in cs:
            continue
        cs.add(oldsteps)
        add_step(x+1, y, oldsteps+1)
        add_step(x-1, y, oldsteps+1)
        add_step(x, y+1, oldsteps+1)
        add_step(x, y-1, oldsteps+1)
    return board

def count_cells(board, steps):
    '''count cells that can be reached in specified nr of steps'''
    res=0
    for cs in board.values():
        if steps in cs:
            res+=1
    return res

sample1=open('data_src/2023-day-21-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
maxsteps=64
sx,sy=find_sx_sy(lines)
board=take_steps(lines, maxsteps, sx, sy)
score=count_cells(board, maxsteps)
print(f'part 1: {score=}')

# part 1: 3795

In [ ]:
# 2023 day 21 part 2 implementation B
# idea: loop through the steps, maintaining data per cell for all 
# instances of that cell together, specifically the active cells for the current step 
# (with instance count per movement map / origin square), and reached count (keeping in mind they
# still have to be reachable on the last step wrt odd/even)
# to determine where next steps can go there are movement maps created for the S position
# and each square along the edge, the movement maps contain next steps for each reachable position 
# in the map
# iffy remaining issues:
# * for now we allow moving to any adjacent instance, including moving back into instances that are
#   already visited, and from two sides moving into the same adjacent instance, 
#   which will cause double counts
# * we only count reachable on steps that are an even nr. of steps removed from the final step, but
#   when w and h are odd many more cells are reachable in a few extra steps (exact conditions to be
#   determined)
# * there is an issue even in the first test set below (sample2, nsteps==6) where reached_count>1
#   for some cells even though there is only once instance involved
# * abandoned, seems to be too compute-intensive to simulate all steps accurately this way

sample2='''
...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
'''

def min_steps(lines, sx, sy):
    '''determine minimum steps to reach each cell, including one step over the edge'''
    board={} # maps (x,y) to minimum steps to reach that cell
    intodo=[ (sx,sy,0), ] # tuples of (x,y,steps-so-far)
    outtodo=[]
    w=len(lines[0]); h=len(lines)

    def add_step(nx, ny, newsteps):
        if nx>=0 and ny>=0 and nx<w and ny<h and lines[ny][nx]=='#':
            return
        intodo.append( (nx, ny, newsteps) )

    while True:
        if len(intodo)<1:
            break
        outtodo.extend(intodo)
        intodo.clear()
        for todotup in outtodo:
            x,y,oldsteps=todotup
            xy=(x,y)
            if xy not in board or board[xy]>oldsteps:
                board[xy]=oldsteps
                if x<0 or x>=w or y<0 or y>=h:
                    continue
                add_step(x+1, y, oldsteps+1)
                add_step(x-1, y, oldsteps+1)
                add_step(x, y+1, oldsteps+1)
                add_step(x, y-1, oldsteps+1)
        outtodo.clear()
    return board

def create_nextstepdata(lines, sx, sy):
    '''create movement maps'''
    w=len(lines[0]); h=len(lines)
    res={}

    def add_all_steps(sx, sy, board):
        for xy, step in board.items():
            nss=res.setdefault( ((sx,sy), xy), set())
            added=set()
            for xy2, step2 in board.items():
                if step2==step+1 and abs(xy2[0]-xy[0])<2 and abs(xy2[1]-xy[1])<2 and \
                 xy2 not in added:
                    nss.add(xy2)
                    added.add(xy2)

    board=min_steps(lines, sx, sy)
    add_all_steps(sx, sy, board)
    for x in range(w):
        for y in range(h):
            if x==0 or x==0 or x==w-1 or y==h-1:
                board=min_steps(lines, x, y)
                add_all_steps(x, y, board)
    return res

def run_steps(sx, sy, w, h, nextstepdata, nsteps):
    '''simulate steps, counting along the way'''
    active_cells={ ((sx,sy), (sx,sy)): 1 } # maps ((originx,originy), (currentx,currenty)) to count of active
    reached_count=collections.Counter() # maps (x,y) to reached&reachable on step nsteps
    if (nsteps-0)%2==0: # count only as reachable if currently reached and even number of steps to the end
        reached_count[(sx,sy)]+=1
    for stepi in range(1, nsteps+1):
        new_active=collections.Counter()
        for oxoyxy, count in active_cells.items():
            oxoy=oxoyxy[0]
            for nextxy in nextstepdata[oxoyxy]:
                nx,ny=nextxy
                if nx<0 or ny<0 or nx>=w or ny>=h: # move to next instance
                    nx=nx%w
                    ny=ny%h
                    new_active[((nx,ny), (nx,ny))]+=count
                else: # same instance
                    new_active[(oxoy, (nx,ny))]+=count
        active_cells=new_active
        print(f'{stepi=}, new {active_cells=}')
        for oxoyxy, count in active_cells.items():
            xy=oxoyxy[1]
            if (nsteps-stepi)%2==0: # count only as reachable if currently reached and even number of steps to the end
                reached_count[xy]+=count
        print(f'{stepi=}, {reached_count=}')
    return sum(reached_count.values())

sample1=open('data_src/2023-day-21-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
w=len(lines[0]); h=len(lines)
print(f'{w=}, {h=}')
sx,sy=find_sx_sy(lines)
nextstepdata=create_nextstepdata(lines, sx, sy) # maps ((originx,originy), (currentx,currenty))
# to a set of (x,y) of next steps (including steps to next instances where x or y can
# be <0 or >=w / >=h)

nsteps=6
score=run_steps(sx, sy, w, h, nextstepdata, nsteps)
print(f'part 2: {score=}')

In [ ]:
# 2023 day 21 part 2 implementation A
# idea: first of all, cells that can be reached in a minimum of n steps can also be reached in n+2*x steps
# next let's start with a bounding box of 11x11 layouts, start in the center, 
# for each cell determine how soon you can reach it, this could lead to insights on layout level

# run an 11x11 layout to see what you can reach when

sample2='''
...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
...........
'''

def tile_steps(lines, tile_factor):
    '''determine minimum steps to reach each cell, after expanding the map
    to tile_factor x tile_factor instances'''
    sx=None; sy=None
    for y,row in enumerate(lines):
        for x,c in enumerate(row):
            if c=='S':
                sx=x; sy=y
    board={} # maps (x,y) to minimum steps to reach that cell
    intodo=[ (sx,sy,0), ] # tuples of (x,y,steps-so-far)
    outtodo=[]
    assert tile_factor % 2 ==1
    w=len(lines[0]); h=len(lines)
    minx=0-w*(tile_factor//2); miny=0-h*(tile_factor//2)
    maxx=minx+w*tile_factor; maxy=miny+h*tile_factor

    def add_step(nx, ny, newsteps):
        if nx<minx or nx>=maxx or ny<miny or ny>=maxy:
            return
        if lines[ny%h][nx%w]=='#':
            return
        intodo.append( (nx, ny, newsteps) )

    while True:
        if len(intodo)<1:
            break
        outtodo.extend(intodo)
        intodo.clear()
        for todotup in outtodo:
            x,y,oldsteps=todotup
            xy=(x,y)
            if xy not in board or board[xy]>oldsteps:
                board[xy]=oldsteps
                add_step(x+1, y, oldsteps+1)
                add_step(x-1, y, oldsteps+1)
                add_step(x, y+1, oldsteps+1)
                add_step(x, y-1, oldsteps+1)
        outtodo.clear()
    return board

sample1=open('data_src/2023-day-21-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
w=len(lines[0]); h=len(lines)
print(f'{w=}, {h=}')
tile_factor=11
board=tile_steps(lines, tile_factor)

# now for each tile we want to know how many steps were reached, and last 'reach' step of that tile
minx=0-w*(tile_factor//2); miny=0-h*(tile_factor//2)
maxx=minx+w*tile_factor; maxy=miny+h*tile_factor

for ty in range(miny, maxy, h):
    for tx in range(minx, maxx, w):
        cnt_even=0
        cnt_odd=0
        lastr=0
        for x in range(tx, tx+w):
            for y in range(ty, ty+h):
                stp=board.get( (x,y) )
                if stp is not None:
                    if stp%2==0:
                        cnt_even+=1
                    else:
                        cnt_odd+=1
                    lastr=max(lastr, stp)
        print(f'[{cnt_even:5d},{cnt_odd:5d},{lastr:5d}]', end=' ')
    print()

# percell={} # maps (x,y) of original layout to a list of steps each instance was first reached
# for xy, minsteps in board.items():
#     sl=percell.setdefault( (xy[0]%w, xy[1]%h), [])
#     sl.append(minsteps)
# for xy, sl in percell.items():
#     assert len(sl)==tile_factor*tile_factor
#     print(xy, ':', sorted(sl))
# conclusion: collecting the data like this makes it very messy, not very useful

In [ ]:
# 2023 day 21 part 2 implementation C
# idea: the resulting area has a diamond shape, we could 'simply' first determine the speed by which 
# the layout can be traversed from one corner to the opposite corner and from side to side, then
# calculate the area of the diamond, calculate the number of layouts which can be completely covered
# and finally for each layout which is only partially covered determine how far cover goes based on steps
# left and cell used for entry.
# (so for each layout we should probably determine from which edge cell we can traverse it fastest, and whether we
# can reach that edge cell the fastest also)


In [ ]:
# TEMPLATE
# 2023 day 21
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2023-day-21-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2023-day-21-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed